# k-NN (k-Nearest Neighbors)

Algoritmo de aprendizaje supervisado basado en la similitud entre observaciones.

Para realizar una predicción, el modelo busca los k puntos más cercanos (con respecto a una métrica impuesta) al nuevo dato y promedia sus valores de salida para estimar el resultado.

Es un enfoque intuitivo, no paramétrico y que no asume una forma funcional específica entre las variables.

Si bien puede ser muy efectivo en datasets pequeños y bien distribuidos, su rendimiento puede verse afectado por la alta dimensionalidad o por una mala elección del parámetro k, por lo que se requiere una cuidadosa elección de sus parámetros.

Carguemos las liberías a utilizar.

In [ ]:
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler, RobustScaler
import pandas as pd
from sklearn.pipeline import Pipeline
from sklearn.neighbors import KNeighborsRegressor
from sklearn.model_selection import train_test_split
from sklearn.compose import ColumnTransformer
from sklearn.decomposition import PCA
from sklearn.preprocessing import OneHotEncoder

Carguemos el dataset a utilizar.

In [ ]:
import kagglehub

# Download latest version
path = kagglehub.dataset_download("harlfoxem/housesalesprediction")

print("Path to dataset files:", path)

Path to dataset files: /kaggle/input/housesalesprediction


In [ ]:
df = pd.read_csv(path + "/kc_house_data.csv")

Separemos las variables entre numéricas y categóricas.

In [ ]:
num = ["sqft_living", "bathrooms", "lat", "long", "sqft_lot", "bedrooms", "yr_built"]
cat = ["zipcode", "waterfront", "condition", "grade"]

In [ ]:
X = df[num + cat]
y = df["price"]

Realicemos la división de los datos entre entrenamiento y validación.

In [ ]:
X_tr, X_te, y_tr, y_te = train_test_split(X, y, test_size=0.2, random_state=42)

Preprocesamiento.

In [ ]:
pre = ColumnTransformer([
    ("num", StandardScaler(), num),
    ("cat", OneHotEncoder(drop="first"), cat)
])

En este caso vamos a utilizar todas las variables por lo que no es necesario incluir un pipeline para hacer reduccion con PCA.

Procedemos a definir el pipeline del metodo k-NN.

In [ ]:
knn = Pipeline([
    ("pre", pre),
    ("knn", KNeighborsRegressor(
        n_neighbors=5, # Numero de vecinos
        weights="distance" # Pesos usados en la prediccion
    ))
])

Entrenamiento.

In [ ]:
knn.fit(X_tr, y_tr)

Pipeline(steps=[('pre',
                 ColumnTransformer(transformers=[('num', StandardScaler(),
                                                  ['sqft_living', 'bathrooms',
                                                   'lat', 'long', 'sqft_lot',
                                                   'bedrooms', 'yr_built']),
                                                 ('cat',
                                                  OneHotEncoder(drop='first'),
                                                  ['zipcode', 'waterfront',
                                                   'condition', 'grade'])])),
                ('knn', KNeighborsRegressor(weights='distance'))])

Definamos las métricas de evaluación del modelo.

In [ ]:
from sklearn.metrics import mean_squared_error as mse
from sklearn.metrics import r2_score
import numpy as np

def rmse(y_true, y_pred):
    return np.sqrt(mse(y_true, y_pred))

Predicción del modelo.

In [ ]:
pred_knn = knn.predict(X_te)

Evaluación del modelo.

In [ ]:
print("RMSE:", rmse(y_te, pred_knn))
print("r2_score:", r2_score(y_te, pred_knn))

RMSE: 178875.112066422
r2_score: 0.7883516234801313


Por ahora, el método no es mejor que los que hemos visto en los otros notebooks.

Ahora, lo que haremos será cambiar los hiperparametros de k-NN para buscar mejorar los errores del metodo.

In [ ]:
knn_8 = Pipeline([
    ("pre", pre),
    ("knn", KNeighborsRegressor(
        n_neighbors=8, # Numero de vecinos
        weights="distance", # Pesos usados en la prediccion
        metric = "cityblock" # Metrica usada para calcular la distancia
    ))
])

knn_8.fit(X_tr, y_tr)
pred_knn_8 = knn_8.predict(X_te)
print("RMSE:", rmse(y_te, pred_knn_8))
print("r2_score:", r2_score(y_te, pred_knn_8))

RMSE: 172818.84159983986
r2_score: 0.8024407888883961


Observemos que cambiando los hiperparametros logramos mejorar el error.

Ahora veamos otro ejemplo con una distancia definida. Podemos utilizar una distancia que entre las variables numericas se utilice la metrica euclideana y entre las variables categoricas se utilice una metrica que vea si son iguales o no en las posiciones.

In [ ]:
def mixed_gower(u, v):
    """
    Example: 'Gower-like' distance for one numeric and one binary dummy.
    Assumes u, v are already scaled the same way.
    """
    # first 7 columns numeric, last 85 columns are 0/1 dummies
    num_part   = np.abs(u[:7] - v[:7]).mean()          # averaged absolute diff
    cat_part   = (u[7:] != v[7:]).mean()               # simple matching
    return 0.7 * num_part + 0.3 * cat_part             # weighted mix

Con lo siguiente podemos ver cuantas variables numericas y categoricas hay luego del pipeline.

In [ ]:
feat_names = pre.get_feature_names_out()

n_total = len(feat_names)
n_num   = sum(name.startswith("num__") for name in feat_names)
n_cat   = sum(name.startswith("cat__") for name in feat_names)

print(f"Total features    : {n_total}")
print(f"Numéricas (num__) : {n_num}")
print(f"Categóricas (cat__): {n_cat}")

Total features    : 92
Numéricas (num__) : 7
Categóricas (cat__): 85


Proceso con metrica definida.

Nota: Se anotaron los resultados debido a que se demora en ejecutar.

In [ ]:
knn_custom = Pipeline([
    ("pre", pre),
    ("knn", KNeighborsRegressor(
        n_neighbors=8,
        weights="distance",
        metric = mixed_gower
    ))
])
knn_custom.fit(X_tr,y_tr)
pred_knn_custom = knn_custom.predict(X_te)
print("RMSE:", rmse(y_te, pred_knn_custom))
print("r2_score:", r2_score(y_te, pred_knn_custom))

# Resultados:
# RMSE : 187949.03937146012
# R² : 0.7663341074607044

Como vimos en veces anteriores los datos tienen outliers, por lo que para el metodo de preparacion utilizaremos RobustScaler.

In [ ]:
pre = ColumnTransformer([
    ("num", RobustScaler(), num),
    ("cat", OneHotEncoder(drop="first"), cat)
])

Proceso pero con el tratamiento de datos outliers.

In [ ]:
knn_8_rs = Pipeline([
    ("pre", pre),
    ("knn", KNeighborsRegressor(
        n_neighbors=8, # Numero de vecinos
        weights="distance", # Pesos usados en la prediccion
        metric = "cityblock" # Metrica usada para calcular la distancia
    ))
])

knn_8_rs.fit(X_tr, y_tr)
pred_knn_8_rs = knn_8_rs.predict(X_te)
print("RMSE:", rmse(y_te, pred_knn_8_rs))
print("r2_score:", r2_score(y_te, pred_knn_8_rs))

RMSE: 171656.83001079227
r2_score: 0.8050885824313354


Mejoramos un poco ya que corregimos los errores de los outliers. Como no eran tantos la mejora no fue significativa pero cuando los outliers son considerables se recomienda realizarlo.